In [0]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools

In [0]:
with open('retail.dat') as f:
    data = [set(i.strip().split()) for i in f]
    data = data[:10000]
    f.close()

In [0]:
data

[{'0',
  '1',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '2',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9'},
 {'30', '31', '32'},
 {'33', '34', '35'},
 {'36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46'},
 {'38', '39', '47', '48'},
 {'38',
  '39',
  '48',
  '49',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58'},
 {'32', '41', '59', '60', '61', '62'},
 {'3', '39', '48'},
 {'63', '64', '65', '66', '67', '68'},
 {'32', '69'},
 {'48', '70', '71', '72'},
 {'39', '73', '74', '75', '76', '77', '78', '79'},
 {'36', '38', '39', '41', '48', '79', '80', '81'},
 {'82', '83', '84'},
 {'41', '85', '86', '87', '88'},
 {'100',
  '101',
  '39',
  '48',
  '89',
  '90',
  '91',
  '92',
  '93',
  '94',
  '95',
  '96',
  '97',
  '98',
  '99'},
 {'36', '38', '39', '48', '89'},
 {'102', '103', '104', '105', '106', '107', '108', '39', '41'},
 {'109', '

In [0]:
all_items = set()
with open('retail.dat','r') as f:
    count=0
    for line in tqdm(f):
        line = set(line.strip().split())
        all_items = all_items.union(line)
        count+=1
        if count==10000:
          break
    f.close()

all_items = list(all_items)

9980it [00:01, 3427.65it/s]


In [0]:
len(all_items)

8600

In [0]:
all_items_set = set()
for i in tqdm(all_items):
    x = set()
    x.add(i)
    x = frozenset(x)
    all_items_set.add(x)

all_items_set

100%|██████████| 8600/8600 [00:00<00:00, 475187.59it/s]


{frozenset({'5386'}),
 frozenset({'296'}),
 frozenset({'1924'}),
 frozenset({'2534'}),
 frozenset({'3271'}),
 frozenset({'2310'}),
 frozenset({'4555'}),
 frozenset({'1395'}),
 frozenset({'8391'}),
 frozenset({'5890'}),
 frozenset({'7681'}),
 frozenset({'5229'}),
 frozenset({'2945'}),
 frozenset({'1874'}),
 frozenset({'3297'}),
 frozenset({'5066'}),
 frozenset({'7328'}),
 frozenset({'5741'}),
 frozenset({'3826'}),
 frozenset({'2221'}),
 frozenset({'2160'}),
 frozenset({'5013'}),
 frozenset({'4935'}),
 frozenset({'2152'}),
 frozenset({'4127'}),
 frozenset({'158'}),
 frozenset({'760'}),
 frozenset({'2250'}),
 frozenset({'7361'}),
 frozenset({'7250'}),
 frozenset({'8308'}),
 frozenset({'6916'}),
 frozenset({'2023'}),
 frozenset({'3818'}),
 frozenset({'1011'}),
 frozenset({'7302'}),
 frozenset({'6527'}),
 frozenset({'5888'}),
 frozenset({'5226'}),
 frozenset({'1839'}),
 frozenset({'7425'}),
 frozenset({'6343'}),
 frozenset({'6860'}),
 frozenset({'8579'}),
 frozenset({'4703'}),
 frozenset({'

In [0]:
len(all_items_set)

8600

In [0]:
min_support = 200
min_confidence = 0.6

In [0]:
def create_Lk(min_support,data,item_sets):
    freq_dict = {}
    n = len(data)
    for i in item_sets:
        for trans in data:
            if i.issubset(trans):
                if i in freq_dict:
                    freq_dict[i] += 1
                else:
                    freq_dict[i] = 1
    
    freq_dict = {i:j for i,j in freq_dict.items() if j>= min_support}
    
    return freq_dict

In [0]:
def printdf(final_table):
  arr = [[], []]
  for i in range(len(list(final_table.items()))):
    arr[0].append(list(final_table.items())[i][0])
    arr[1].append(list(final_table.items())[i][1])
  
  df = pd.DataFrame(np.array(arr))
  return df

In [0]:
def apriori(data,item_sets,min_support,min_confidence):
    L_table_list = {}
    count = 1
    L_table_list[count] = create_Lk(min_support,data,item_sets)
    L = L_table_list[1]
    count += 1
    
    while len(L)!=0:
        new_item_set = set(L.keys())
        new_item_set = set([i.union(j) for i in new_item_set for j in new_item_set if len(i.union(j))==count])
        L = create_Lk(min_support,data,new_item_set)
        L_table_list[count] = L
        
        count += 1

        if len(L)!=0:
          final_table = L

        print("Length of table reduced to: "+str(len(L)))

    print(final_table)
    
    return final_table, L_table_list

final_table, L_table_list = apriori(data,all_items_set,min_support,min_confidence)

Length of table reduced to: 26
Length of table reduced to: 10
Length of table reduced to: 2
Length of table reduced to: 0
{frozenset({'32', '48', '39', '41'}): 282, frozenset({'48', '39', '41', '38'}): 315}


In [0]:
df = printdf(final_table)
df

,0,1
0,"(32, 48, 39, 41)","(48, 39, 41, 38)"
1,282,315


In [0]:
possible_combinations = {}
for item_set, support_count in final_table.items():
  possible_combinations[item_set] = list()
  for sz in range(1, len(item_set)):
    for comb in itertools.combinations(item_set, sz):
        possible_combinations[item_set].append(comb)
    
print(possible_combinations)

{frozenset({'32', '48', '39', '41'}): [('32',), ('48',), ('39',), ('41',), ('32', '48'), ('32', '39'), ('32', '41'), ('48', '39'), ('48', '41'), ('39', '41'), ('32', '48', '39'), ('32', '48', '41'), ('32', '39', '41'), ('48', '39', '41')], frozenset({'48', '39', '41', '38'}): [('48',), ('39',), ('41',), ('38',), ('48', '39'), ('48', '41'), ('48', '38'), ('39', '41'), ('39', '38'), ('41', '38'), ('48', '39', '41'), ('48', '39', '38'), ('48', '41', '38'), ('39', '41', '38')]}


In [0]:
def check(comb1, comb2, L_table_list, min_confidence):
  union_comb = (comb1.union(comb2))
  p1 = L_table_list[len(union_comb)][union_comb]
  p2 = L_table_list[len(comb2)][comb2]

  p = float(p1/p2)

  if(p>=min_confidence):
    return True
  return False

In [0]:
assoc_rules = {}
for item_set, combinations in possible_combinations.items():
  assoc_rules[item_set] = []
  for i in range(len(combinations)):
    for j in range(len(combinations)):

      comb1 = frozenset(combinations[i])
      comb2 = frozenset(combinations[j])

      if(comb1.union(comb2)==item_set and  comb1.isdisjoint(comb2)):
        rule = {}

        if check(comb1, comb2, L_table_list, min_confidence):
          rule[comb1] = comb2
          assoc_rules[item_set].append(rule)


In [0]:
for item_set, rules in assoc_rules.items():
  print(item_set)
  print("--------------------------------------------")

  for rule in rules:
    key = list(rule.keys())[0]
    val = list(rule.values())[0]

    print(str(key)+"----->"+str(val))

  print("\n\n")

frozenset({'32', '48', '39', '41'})
--------------------------------------------
frozenset({'48'})----->frozenset({'41', '32', '39'})
frozenset({'39'})----->frozenset({'41', '32', '48'})



frozenset({'48', '39', '41', '38'})
--------------------------------------------
frozenset({'39'})----->frozenset({'41', '48', '38'})



